In [1]:
#Import libraries
from pymeasure.instruments.lighthousephotonics import Sprout #import laser codes
laser_power = Sprout('COM4')
laser_power.adapter.connection.baud_rate = 19200
laser_power.adapter.connection.read_termination = '\r'
laser_power.adapter.connection.write_termination = '\r'
laser_power.power = 0.01
import time
from datetime import timedelta
import rpy2.robjects as robjects

from newportxps import NewportXPS #motion controller

from threading import Thread
# from thread import MotionThread, MirrorThread
import os, sys

import numpy as np
import pandas as pd
from write_utils import write_data_files, get_mean, get_move_y, duplicate_to_dataset

from flipper import mirror #import flipper mirror codes
from spectra import capture_photo #import spectrometer codes

from pathlib import Path


c:\Users\UWAdmin\anaconda3\lib\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)
c:\Users\UWAdmin\anaconda3\lib\site-packages\pyvisa\highlevel.py:3355: FutureWarning: get_instrument is deprecated and will be removed in 1.12, use open_resource instead.
  FutureWarning,


In [2]:
#Define the file you want to read
path = Path(r'c:\\Users\\UWAdmin\\Desktop\\_pyControl\\campaigns')   #changes directory to new file created in next block. run this, run next block, update campaign_path, and run this again
campaign_path = path / "2023-06-26-BATCH-1"
# os.mkdir(campaign_path)
os.chdir(campaign_path)

In [3]:
#Create new file (only need to run this when starting a new series of experiments)
write_data_files(path, series=1, nr_random_lines=6, \
    power_range=(300,2600), time_range=(2000, 28000), pressure_range=(60,800),passes_range=(1,2),defocus_range=(-2.5,0), batch=True)

# set passes_range=(1,2) to set number of passes to 1 for all lines
# values here need to be the same as values in red ML section

TODAY's DATE: 2023-08-09
POWER: (300, 2600)
 TIME: (2000, 28000)
 PRESSURE: (60, 800)
Here is the new Campaign Folder:2023-08-09-BATCH-1


In [16]:
#Connect to XPS Device
XPSD_REMOTE_IP = '192.168.254.254'
xps = NewportXPS(XPSD_REMOTE_IP)
print(xps.status_report())
#pos_all() to get position of all axes
def pos_all():
    for sname, _ in xps.stages.items():
        print('{}: {}'.format(sname, xps.get_stage_position(sname)))
xps.kill_group('XYZ')
xps.initialize_allgroups()
xps.home_allgroups()

# XPS host:         192.168.254.254 (XPS-1B81)
# Firmware:         XPS-D-N13019
# Current Time:     Thu Aug 10 14:18:38 2023
# Last Reboot:      Wed May 17 10:08:36 2023
# Trajectory Group: XYZ
# Groups and Stages
XYZ (multipleaxesinuse), Status: Not initialized state due to an emergency brake : see positioner error
   XYZ.X (MFA@MFA-CC@XPS-DRV11)
      Hardware Status: ZM low level
      Positioner Errors: OK
   XYZ.Y (MFA@MFA-CC@XPS-DRV11)
      Hardware Status: ZM low level
      Positioner Errors: OK
   XYZ.Z (MFA@MFA-CC@XPS-DRV11_Z)
      Hardware Status: Minus end of run activated - ZM high level
      Positioner Errors: OK


In [5]:
#Starting Lightfield Raman Software
capture_photo("start",2,1,0) 


In [6]:
#Define pressure functions
from pressure import close_valve,open_valve,close_all,current_pressure,gopr, to_ambient, to_vacuum, quick_fill
current_pressure()
# gopr(500)

c:\Users\UWAdmin\anaconda3\lib\site-packages\pyvisa\highlevel.py:3355: FutureWarning: get_instrument is deprecated and will be removed in 1.12, use open_resource instead.
  FutureWarning,


112

Remember to change z-axis value

In [17]:
#Set z axis value
z_measure=4.32
xps.move_stage('XYZ.Z', z_measure)
xps.move_stage('XYZ.X', 4.5)
xps.move_stage('XYZ.Y', 12)
pos_all()

XYZ.X: 4.5000016524
XYZ.Y: 11.9998746108
XYZ.Z: 4.3199987544


In [8]:
#Define Parallelism Threads
class MotionThread(Thread):
   
    def run(self):
        import pandas as pd
        d = pd.read_csv('foreward.trj',header=None) # scantime 2 (see segment 1 or row1)
        d = d.loc[:, (d != 0).any(axis=0)] # remove last 3 cols with 0
        d.columns = ['ramptime','rampdist','rampvel']
        total_time=np.sum(d['ramptime'])
        
        if d['ramptime'][0]<0.5:
            time.sleep(.5-d['ramptime'][0])
        print("MOTION THREAD\n")
        print("XPS run trajectory")
        xps.run_trajectory('foreward',)
        print('Total time {} in class'.format(total_time))
        time.sleep(total_time)
        print("TOTAL_TIME DONE, LASER 0.5W")
        # laser_power.power=0.1
        print("finished and current position is:\n")
        pos_all()

class MirrorThread(Thread):

    def run(self):
        import pandas as pd
        d = pd.read_csv('foreward.trj',header=None) # scantime 2 (see segment 1 or row1)
        d = d.loc[:, (d != 0).any(axis=0)] # remove last 3 cols with 0
        d.columns = ['ramptime','rampdist','rampvel']
        mirror_sleep=d['ramptime'][0]-.5
        print("MIRROR THREAD")
        for i in range(2):
            if i==0:
                print(d['ramptime'][0])
                if d['ramptime'][0]<0.5:
                    print("mirror on")
       
                    start_time = time.monotonic()
                    mirror('on')
       
                    end_time = time.monotonic()
                    print(timedelta(seconds=end_time - start_time))
                    time.sleep(d['ramptime'][1]) #time for linear line    

                else:
                    print('{} > 0.5'.format(d['ramptime'][0]))
                    print('Mirror sleep {} in class'.format(mirror_sleep))
                    time.sleep(mirror_sleep)
                
                    print("mirror on")
                    start_time = time.monotonic()
                    mirror('on')
                    end_time = time.monotonic()
                    print(timedelta(seconds=end_time - start_time))
                    time.sleep(d['ramptime'][1]) #time for linear line
            else:
                mirror('off')
                print("mirror off")

Define MBO

In [9]:
robjects.r('''
experiment<-function(s){

  presToR <- function (x) {(x - 50) / 10}
  presToExp <- function (x) {x * 10 + 50}
  defocusToR <- function (d) {d*100}
  defocusToExp <- function (d) {d/100}

  if(s==1){ #BO called first time to initialize and propose first point
  require(XML)

    presToR <- function (x) {(x-50)/10}
    presToExp <- function (x) {x*10+50}
	defocusToR <- function (d) {d*100}
	defocusToExp <- function (d) {d/100}

    suppressWarnings({suppressMessages({library(mlrMBO)
                                        library(ggplot2)
                                        library(doMC)
                                      })})

    ps = makeParamSet(
      makeIntegerParam("power", lower = 100, upper = 2600),                        
      makeIntegerParam("time", lower = 2000, upper = 28000),
      makeIntegerParam("pressure", lower = 1, upper = 75, trafo = function(x) x * 10 + 50),
      makeIntegerParam("defocus", lower = -300, upper = 0, trafo = function(x) x/100)
    )

    ctrl = makeMBOControl(y.name = "resistance", propose.points = 6L)
    ctrl = setMBOControlInfill(ctrl, opt = "focussearch", 
                              opt.focussearch.maxit = 10,
                              opt.focussearch.points = 10000,
                              crit = makeMBOInfillCritEI())

    ctrl = setMBOControlMultiPoint(ctrl, method = "cl", cl.lie = min)
    ctrl = setMBOControlTermination(ctrl, iters = 1)

    data = read.csv("data.csv", colClasses = c(NA, NA, NA, NA, "NULL", NA))
    data$pressure <- presToR(data$pressure)
	data$defocus <- defocusToR(data$defocus)

    suppressMessages({opt.state = initSMBO(par.set = ps, 
                                          design = data,
                                          control = ctrl, 
                                          minimize = TRUE, 
                                          noisy = TRUE)})
    print("Proposed parameters:")
    prop = suppressWarnings({proposePoints(opt.state)})
    print(prop$prop.points)
    print(prop$crit.components)
    saveRDS(prop,'./prop.rds')
    saveRDS(opt.state,'./opt.state.rds')
    dummy_ratio = 0
    dummy_resistance = 0
    cat("Expected value (upper bound):\n")
    cat(paste(prop$crit.components$mean, " (",
              prop$crit.components$mean + prop$crit.components$se, ")\n", sep = ""))

    prop$prop.points$pressure <- presToExp(prop$prop.points$pressure)
	  prop$prop.points$defocus <-defocusToExp(prop$prop.points$defocus)

    x <- data.frame(prop$prop.points, dummy_ratio, dummy_resistance,
                    prop$crit.components$mean,
                    prop$crit.components$mean + prop$crit.components$se, 
                    2*prop$crit.components$mean + prop$crit.components$se)

    x2 <- c(prop$prop.points)

    write.table(x2, file = "data.csv", sep = ",",
                append = TRUE, quote = FALSE,col.names = FALSE, row.names = FALSE)

    write.table(x, file = "plot_data.csv", sep = ",",
                append = TRUE, quote = FALSE,col.names = FALSE, row.names = FALSE)

  }
  else{
    
    presToR <- function (x) {(x-50)/10}
    presToExp <- function (x) {x*10+50}
	defocusToR <- function (d) {d*100}
	defocusToExp <- function (d) {d/100}

    prop=readRDS('./prop.rds')
    opt.state=readRDS('./opt.state.rds')
    library(mlrMBO)
    library(R.utils)
    print(prop$crit.components)
    print(prop$prop.points)

    print("Proposed parameters:")
    l2keep <- 6
    nL <- countLines("data.csv")
    s <- read.csv("data.csv", header=FALSE, skip=nL-l2keep)['V5']
    updateSMBO(opt.state, x = prop$prop.points, y = as.list(s$V5))
    prop <- suppressWarnings({proposePoints(opt.state)})
    print(prop$prop.points)
    print(prop$crit.components)
    saveRDS(prop,'./prop.rds')
    saveRDS(opt.state,'./opt.state.rds')

    dummy_ratio = 0
    dummy_resistance = 0
    cat("Expected value (upper bound):\n")
    cat(paste(prop$crit.components$mean, " (",
              prop$crit.components$mean + prop$crit.components$se, ")\n", sep = ""))

    prop$prop.points$pressure <- presToExp(prop$prop.points$pressure)
	prop$prop.points$defocus <-  defocusToExp(prop$prop.points$defocus)


    x <- data.frame(prop$prop.points, dummy_ratio, dummy_resistance,
                    prop$crit.components$mean,
                    prop$crit.components$mean + prop$crit.components$se, 
                    prop$crit.components$mean + prop$crit.components$mean + prop$crit.components$se)
					
    x2<-c(prop$prop.points)
	print("TEST: Here are the proposed uncertainty")
	print(x)
    write.table(x2, file = "data.csv", sep = ",",
                append = TRUE, quote = FALSE,col.names = FALSE, row.names = FALSE)

    write.table(x, file = "plot_data.csv", sep = ",",
                append = TRUE, quote = FALSE,col.names = FALSE, row.names = FALSE)

  }}

  ''')
  


<rpy2.robjects.functions.SignatureTranslatedFunction object at 0x00000216C0D711C8> [RTYPES.CLOSXP]
R classes: ('function',)

# (L) Propose next 8 samples

In [10]:
#Define running files
import pandas as pd

def _printing_line_count(data:pd.DataFrame, measured_value:str = 'resistance'):
    printed_lines_count = len(data['resistance'].dropna(inplace=False))
    return printed_lines_count

def _first_line_to_print(col:int=0, row:int=0):
    None
    
def _lines_to_print(data:pd.DataFrame, line_count:int):
    printed_count = _printing_line_count(data)
    print_line_set = data.loc[printed_count::]
    return print_line_set

def get_line_numbers(data:pd.DataFrame, printing_line_count:int, measured_value:str='resistance'):
    printed_lines = _printing_line_count(data, measured_value)
    line_numbers = list(range(printed_lines, printed_lines+printing_line_count))
    return line_numbers

def get_line_parameters(data:pd.DataFrame, printing_line_count:int):
    parameters:pd.DataFrame = data.loc[get_line_numbers(data,printing_line_count)]
    return parameters.reset_index()
##BELOW BLOCK IS WHERE STARTING LINE COORDINATES CAN BE CHANGED
def generate_line_coordinates(line_count:int, y_start_location:float=6.5, x_start_location:list = [1.5, 4.5]):
    lines_per_column = int(line_count/2)
    y_stop_location: float = 11
    y_coordinates = np.linspace(y_start_location, y_stop_location, lines_per_column)
    coordinates = []
    for x in x_start_location:
        for y in y_coordinates:
            coordinates.append((x,y))

    return coordinates

data = pd.read_csv("data.csv")
rsum = robjects.r['experiment']


NR_LINES_IN_SAMPLE = _printing_line_count(data)
_NR_PROPOSED_LINES = 6 # don't change this




In [11]:
# Don't run this section on first sample (iteration) of new campaign

if len(data['resistance']) == NR_LINES_IN_SAMPLE:
    # print(len(d['resistance']))
    print(f"First batch proposal with {NR_LINES_IN_SAMPLE} lines to train")
    rsum((1))
    duplicate_to_dataset(nr_proposed_lines=_NR_PROPOSED_LINES, prepattern=False)

elif len(data['resistance']) < NR_LINES_IN_SAMPLE:
    print ("Not enough data points in dataset. Exiting.")
    exit()

else:
    print(f"Further batch proposals of {NR_LINES_IN_SAMPLE} lines")
    proposed = data['resistance'].iloc[-NR_LINES_IN_SAMPLE:]
    rsum((list(proposed)))
    duplicate_to_dataset(nr_proposed_lines=_NR_PROPOSED_LINES)

R[write to console]: Loading required package: XML



First batch proposal with 234 lines to train


R[write to console]: 
Attaching package: 'XML'


R[write to console]: The following object is masked from 'package:tools':

    toHTML




[1] "Proposed parameters:"
     power time pressure defocus
163   1682 2000       41     -16
462   1763 7737       23     -49
695    464 6194       32     -20
1105   632 6338       49     -50
1612  1810 4664       15     -52
24     595 4060       67     -13
        se      mean
1 10455783  -1802547
2 12614763 -29034498
3 11527259 -41540175
4 10461832 -28230603
5 10792379 -22482954
6 13761543  -1752277
Expected value (upper bound):
-1802546.87417269 (8653236.43516086)
 -29034498.0416606 (-16419734.5563889)
 -41540175.3961547 (-30012916.6705247)
 -28230603.235539 (-17768771.164067)
 -22482953.5101342 (-11690574.6552935)
 -1752277.21903786 (12009265.9183825)


In [12]:
#View experiment values
pd.read_csv("data.csv").tail(6)


,power,time,pressure,defocus,ratio,resistance
234,1682,2000,460,-0.16,NaN,NaN
235,1763,7737,280,-0.49,NaN,NaN
236,464,6194,370,-0.20,NaN,NaN
237,632,6338,540,-0.50,NaN,NaN
238,1810,4664,200,-0.52,NaN,NaN
239,595,4060,720,-0.13,NaN,NaN


# (M) Begin the experiment

In [13]:

#Define experiment operations
measurement_times = []
import time
print(f'Here is the last group and next group of data points:\n{data}')


def batch_experiment(total_lines) -> None:

    print("Loading data from csv")
    data:pd.DataFrame = pd.read_csv('data.csv')
    print("Batch Experiment is starting")
    _mirror_move_time:float = .5 # in seconds
    _line_length:float = 2 # in mm
    vacuum_time:float = 2 # in minutes
    line_coordinates:list = generate_line_coordinates(total_lines)
    line_parameters:list = get_line_parameters(data, total_lines)
    line_numbers:list = get_line_numbers(data, total_lines)

    _spot_measured = 9
    TARGET:str = 'GD'
    print(f"Here's the number of total lines:{total_lines}")
    if (len(line_coordinates) != len(line_parameters)) | (len(line_coordinates) != len(line_numbers)):
        print('Print variables do no match')
        exit()

    for i in range(total_lines):
        line_start_time = time.time()
        laser_power.power = 0.01
        save_line:int = line_numbers[i]
        parameters:pd.core.series.Series = line_parameters.loc[i]

        print(f"====== Printing line #{i+1}/{total_lines}")
        print(f"Saving to line {save_line}")

        print(f"\nCurrent patterning parameters are:\n {parameters}")
        time_of_file:float = parameters['time']
        power_of_file:float = parameters['power']
        pressure_of_file:float = parameters['pressure']
        number_of_passes = 1
        defocus:float = parameters['defocus']

        line_i_coorinates:tuple = line_coordinates[i]
        line_start_x_coordinate:float = line_i_coorinates[0]
        line_i_y_coordinate:float = line_i_coorinates[1]
        xps.move_stage('XYZ.X',line_start_x_coordinate)
        xps.move_stage('XYZ.Y',line_i_y_coordinate)
        pos_all()
        print("\n\n")
        data:pd.DataFrame = pd.read_csv('data.csv')
        plot_data_df:pd.DataFrame = pd.read_csv('plot_data.csv')

        print("Vacuum-infill in progress...")
     
        to_ambient()
        print(f"1st ambient: {current_pressure()}")
        time.sleep(1)
        to_ambient()
        print(f"2nd ambient: {current_pressure()}")
        time.sleep(1)
        to_ambient()
        print(f"3rd ambient: {current_pressure()}")
        time.sleep(1)
        laser_power.write("OPMODE=Off")
        
        to_vacuum(vacuum_time)
        time.sleep(1)
        print(f"vac : {current_pressure()}")

        laser_power.power = 0.01

        gopr(pressure_of_file)
        
        # for j in range(3):
        #     # print(f"Pressure count {j}")
        #     gopr(pressure_of_file)
        #     close_all()
        #     time.sleep(5)

        laser_power.write("OPMODE=On")
        print(f"\n Pressure of file is :{pressure_of_file}\n")
        print(f"\n Pressure is now :{current_pressure()}\n")

        xps.define_line_trajectories(start=float(line_start_x_coordinate),
                                 stop=float(line_start_x_coordinate)+_line_length,
                                 step=0.01,
                                 scantime=(time_of_file/1000),
                                 axis='X')

        xps.download_trajectory('foreward.trj')

        d = pd.read_csv('foreward.trj', header=None)
        d = d.loc[:, (d != 0).any(axis=0)]
        d.columns = ['ramptime','rampdist','rampvel']
        print(d)

        total_time = np.sum(d['ramptime'])
        mirror_sleep = d['ramptime'][0] - _mirror_move_time
        print(f'Total time: {total_time}, Mirror_sleep: {mirror_sleep} in MAIN')

        mirror("off")
        pp=power_of_file
        laser_power.power = (pp/0.2163)/1000
        print("Power is now: {} mW".format(power_of_file))
        print("Allowing 60 seconds for laser to ramp")
        time.sleep(60)
        print("Starting Patterning")
        # PATTERN EXPERIMENT
        print("Defocusing")
        xps.move_stage('XYZ.Z',float(z_measure + defocus))
        pos_all()
        print(f'Number of Passes to Perfrom:{number_of_passes}')
        for i in range(number_of_passes):
            print(f"This is pass {i+1}/{number_of_passes}")
            #multi pattern
            xps.move_stage('XYZ.X',(float(line_start_x_coordinate) - d['rampdist'][0]))
            print('motors move to start location')
            pos_all()
            time.sleep(1)
            print("\n\n")
            
            print("###### Starting pattern in 2 seconds! #########")
            ##Draw Lines
            time.sleep(1)
            
            motion_thread = MotionThread()
            mirror_thread = MirrorThread()

            motion_thread.start()
            mirror_thread.start()

            motion_thread.join()
            mirror_thread.join()
            
            mirror('off')
            time.sleep(1)
            print("\n Pass done\n")
        
        ##raman spectra analysis
        laser_power.power=0.1
        time.sleep(1)

        line_stop_time = time.time()
        total_time = line_stop_time - line_start_time
        print(f"Line took: {total_time}")
        measurement_times.append(total_time)
        print(f"Measurement Times = {measurement_times}")
    ########################################

    # post patterning raman analysis
        print("Refocusing")
        xps.move_stage('XYZ.Z',float(z_measure))
        pos_all()
        check=[float(line_start_x_coordinate) + .2*_line_length, \
                float(line_start_x_coordinate) + .4*_line_length, \
                float(line_start_x_coordinate) + .6*_line_length]
        # mm=0
        for spot in range(3):
            print(f"############### SPOT {spot} ##################")
            xps.move_stage('XYZ.X',check[spot])
            print(pos_all())

            for iii in range(1):  #change number in parenthesis to set number of raman scans per spot
                print(f"############# iteration {iii} ###############")
                mirror("on")
                #writing G/D of 3 spots in 3 lines inside dataset.csv
        #         capture_photo("on",spot,line)
                print(f"Measuring patterned line for {TARGET}")
                GD = capture_photo("on", spot, save_line, iii, target=TARGET)

                mirror("off")
                time.sleep(5)

        # if save_line <= TOTAL_LINES:
        # print(f"############# LINE {save_line} <= {TOTAL_LINES}, CURRENT_STEP={steps}, TAKING MEAN: #############")
        # get_mean(steps, save_line, _spot_measured)




Here is the last group and next group of data points:
     power   time  pressure  defocus  ratio    resistance
0      700  18935       780    -0.85    NaN  4.776796e+07
1     1990   6214       170    -0.09    NaN  6.318206e-01
2     1386  25838       100    -0.92    NaN  4.776796e+07
3      847  15760       250    -0.07    NaN  4.776796e+07
4     1868  27739       450    -0.54    NaN  4.538926e+03
..     ...    ...       ...      ...    ...           ...
229   1557   6459       550    -0.12    NaN  1.624695e+00
230   1761   6405       360    -0.12    NaN  4.846729e-01
231    117   2618        60    -0.57    NaN  4.776796e+07
232   1698   2000       540    -0.15    NaN  5.460515e-01
233   1829   3890       610    -0.14    NaN  6.000782e-01

[234 rows x 6 columns]


In [14]:
#Run experiment
intensity_check =capture_photo('adjust',0,0,0)
print("---Final Verification of Batch Experiment---")
print("Current Pressure: ", current_pressure())
pos_all()
print(f'Here is the intensity:{intensity_check}')
print("-----------------------------------------")
print("-----------------------------------------")

try:
        laser_power.write("OPMODE=On")
        batch_experiment(total_lines=6)
except:
        to_ambient()
        laser_power.write("OPMODE=Off")
        sys.exit()
laser_power.write("OPMODE=Off")

MIRROR UP
 
MIRROR DOWN
---Final Verification of Batch Experiment---
Current Pressure:  111
XYZ.X: 4.5000016524
XYZ.Y: 11.9999985066
XYZ.Z: 4.3199987544
Here is the intensity:440.3333333333333
-----------------------------------------
-----------------------------------------
Loading data from csv
Batch Experiment is starting
Here's the number of total lines:6
====== Printing line #1/6
Saving to line 234

Current patterning parameters are:
 index          234.00
power         1682.00
time          2000.00
pressure       460.00
defocus         -0.16
ratio             NaN
resistance        NaN
Name: 0, dtype: float64
XYZ.X: 1.5000064506
XYZ.Y: 6.5001577482
XYZ.Z: 4.3199987544



Vacuum-infill in progress...
1st ambient: 34
2nd ambient: 22
3rd ambient: 20
10
9
9
8
9
9
8
9
9
8
9
9
8
9
8
8
8
8
9
9
9
9
8
9
9
9
9
8
9
9
9
9
8
9
8
9
8
8
9
8
8
8
8
8
9
9
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
8
7
8
8
8
8
8
8
8
8
8
8
8
8
7
8
8
8
8
8
8
7
8
8
7
8
8
7
7
7
8
8
8
8
8
8
8
8
7
8
7
8
7
8
8
8
8
7
7
7
7
7
8
8
8
8
8


In [15]:
#De-pressurize line and move system to default position
to_ambient()
to_ambient()
# to_vacuum(0.01)
laser_power.write("OPMODE=Off")
xps.kill_group('XYZ')
xps.initialize_allgroups()
xps.home_allgroups()
current_pressure()

24

In [ ]:
print(measurement_times)